Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

In [24]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import lightgbm as lgb
from lightgbm import LGBMModel,LGBMClassifier

In [65]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [11]:
# print out data sample
# data

In [13]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [15]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

# the pipeline puts together a bunch fit then transform, fit then predict.

In [16]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   48.3s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  2.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [17]:
grid_search.best_score_

0.8867877056983545

In [18]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1], dtype=int64)

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [21]:
#from sklearn.utils.fixes import loguniform

In [25]:
import pandas as pd
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [45]:
train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


### Define Pipeline Components

In [5]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

# categories = ['Scotch', 'Bourbon/Tennessee', 'Craft Whiskey', 'Canadian']

# data = train(subset='train', categories=categories)

pipe = Pipeline([('vect', vect), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [6]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'clf__max_depth':(10,20,40),
    'clf__n_estimators':(10, 15)
}

random_search = RandomizedSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
random_search.fit(train.description, train.category)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   48.3s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:   54.4s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                        

In [7]:
random_search.best_score_

0.8631083694913482

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [12]:
# Predictions on test sample
pred = random_search.predict(test['description'])

In [13]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [14]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,1
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [15]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('submission2.csv', index=False)

## Challenge

You're trying to achienve 90% Accuracy on your model.

In [17]:
xgb= XGBClassifier()
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))

pipe = Pipeline([('vect', vect), ('clf', xgb)])

In [18]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'clf__max_depth':(5,10,15,20),
    'clf__n_estimators':(10, 15)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(train.description, train.category)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   27.3s
[Parallel(n_jobs=4)]: Done 160 out of 160 | elapsed:  2.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [71]:
grid_search.best_score_

0.9124643002855976

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [35]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

In [67]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    # these are percents 90% 95% 100%
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators': (5,10,20)
}

In [68]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [ ]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=4, verbose=1)
grid_search.fit(data.data, data.target)

In [14]:
grid_search.best_score_

0.8786464410735122

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [51]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

In [52]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
lgb = LGBMClassifier(random_state=64)

In [53]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])

# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', lgb)])

In [54]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'clf__n_estimators': (5,10,20),
    'clf__max_depth':(5,10,15,20)
}


In [55]:
# fit
random_search = RandomizedSearchCV(pipe,parameters, cv=6, n_jobs=2, verbose=1)
random_search.fit(train.description, train.category)

Fitting 6 folds for each of 10 candidates, totalling 60 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  3.7min
[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:  5.8min finished


RandomizedSearchCV(cv=6, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

In [56]:
random_search.best_score_

0.9060324825986079

In [28]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'lsi__vect__min_df':[.02, .05],
    'clf__n_estimators': (5,10,20),
    'clf__max_depth':(5,10,15,20)
}

In [29]:
# fit
random_search = RandomizedSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
random_search.fit(train.description, train.category)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   33.3s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:   54.3s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

In [30]:
random_search.best_score_

0.8774127540084986

In [31]:
# fit
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(train.description, train.category)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   19.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  5.0min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 10.4min
[Parallel(n_jobs=4)]: Done 1080 out of 1080 | elapsed: 14.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [32]:
grid_search.best_score_

0.8801176971389738

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [20]:
# Predictions on test sample
pred = random_search.predict(test['description'])

In [21]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [22]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,3
2,1390,1
3,1024,1
4,1902,1


In [24]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('submission3.csv', index=False)

In [57]:
# new attempt
pred = random_search.predict(test['description'])

In [58]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [59]:
submission.to_csv('submission4.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [15]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [20]:
doc = nlp("Two bananas in pyjamas")

In [21]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [26]:
len(X) == len(data.data)

True

In [22]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [23]:
X = get_word_vectors(data.data)

rfc.fit(X, data.target)

/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(X, data.target)

0.9918319719953326

## Follow Along

In [26]:
nlp = spacy.load("en_core_web_lg")
tokenizer = spacy.tokenizer.Tokenizer(nlp.vocab)
lgb = LGBMClassifier(random_state=64)
xgb = XGBClassifier()

In [27]:
train.head(10)

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2
5,12,Distilled at the now legendary Stitzel-Weller ...,2
6,14,"No age statement, but distilled in 1998. A bea...",2
7,15,"Monarch, the 75th anniversary limited edition ...",4
8,17,"The complete package: uncut, unfiltered, full-...",2
9,21,"A marriage of four different bourbons, ranging...",2


In [28]:
STOPWORDS = nlp.Defaults.stop_words.union(['whiskey','bottle'])

def tokenizing(train):

    """ create tokenizing lemma w/o stopwords and w/o punctuation"""
    tokens = []
    for doc in tokenizer.pipe(train.description):
        doc_tokens = [token.lemma_ for token in doc if (token.text not in STOPWORDS) and (token.is_punct == False)]
        tokens.append(doc_tokens)
    train['tokens'] = tokens
    return train
    
train = tokenizing(train) 

In [29]:
train.head()

,id,description,category,tokens
0,1,A marriage of 13 and 18 year old bourbons. A m...,2,"[A, marriage, 13, 18, year, old, bourbons., A,..."
1,2,There have been some legendary Bowmores from t...,1,"[There, legendary, Bowmores, mid-60s, bite, eq..."
2,3,This bottling celebrates master distiller Park...,2,"[This, bottle, celebrate, master, distiller, P..."
3,4,What impresses me most is how this whisky evol...,1,"[What, impress, whisky, evolves;, it's, incred..."
4,9,"A caramel-laden fruit bouquet, followed by une...",2,"[A, caramel-laden, fruit, bouquet,, follow, un..."


In [30]:
def transform_string(tokens):
    return ' '.join(tokens)

train['tokens'] = train.tokens.apply(transform_string)

In [31]:
train.head()

,id,description,category,tokens
0,1,A marriage of 13 and 18 year old bourbons. A m...,2,A marriage 13 18 year old bourbons. A mature e...
1,2,There have been some legendary Bowmores from t...,1,There legendary Bowmores mid-60s bite equal. A...
2,3,This bottling celebrates master distiller Park...,2,This bottle celebrate master distiller Parker ...
3,4,What impresses me most is how this whisky evol...,1,What impress whisky evolves; it's incredibly c...
4,9,"A caramel-laden fruit bouquet, followed by une...",2,"A caramel-laden fruit bouquet, follow unending..."


In [32]:
docs = train['tokens']

In [33]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [18]:
X = get_word_vectors(train.tokens)

lgb.fit(X, train.tokens)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=64, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [19]:
lgb.score(X, train.tokens)

0.5166279969064191

In [36]:
vect = TfidfVectorizer(stop_words='english')
svd = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=10)

In [41]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])

# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', xgb)])

In [45]:
params = {
    'lsi__vect__max_df' : (.75, 1),
    'lsi__vect__min_df' : (.02, .05),
    'lsi__vect__max_features': (500,1000),
    'clf__n_estimators': (10,50,100),
    'clf__max_depth':(5,10,20),
}

random_search = RandomizedSearchCV(pipe, param_distributions=params, cv=5, n_jobs=2, verbose=1)

In [46]:
random_search.fit(train.tokens, train.category)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  2.4min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:  2.4min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

In [47]:
random_search.best_score_

0.8727668536179174

In [49]:
random_search.fit(train.description, train.category)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:  2.4min
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:  2.5min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

In [50]:
random_search.best_score_

0.8932630336885656

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on